# 协程
- 参考资料
    - http://python.jobbole.com/86481/
    - http://python.jobbole.com/87310/
    - https://segmentfault.com/a/1190000009781688
    
# 迭代器
- 可迭代(Iterable):直接作用于for循环的变量
- 迭代器(Iterator):不但可以作用于for循环，还可以被next调用
    - 迭代器都是可迭代的,但可迭代的不全是迭代器
- list是典型的可迭代对象，但不是迭代器
- 通过isinstance判断
- iterable和iterator可以转换
    - 通过iter函数

In [3]:
# 可迭代
l=[i for i in range(10)]

#l是可迭代的,但是不是迭代器
for i in l:
    print(i,end=' ')
    
#range是迭代器    
for i in range(5):
    print(i,end=' ')

0 1 2 3 4 5 6 7 8 9 0
1
2
3
4


In [4]:
#isinstance案例
#判断某个变量是狗屎一个实例

#判断是否可迭代
from collections import Iterable
l1=[1,2,3,4,5]

#判断是够可迭代
print(isinstance(l1,Iterable))

#判断是否是迭代器
from collections import Iterator
print(isinstance(l1,Iterator))

True
False


In [9]:
#iter函数
s='i love you'
print(isinstance(s,Iterable))
print(isinstance(s,Iterator))
print('* '*20)
s_iter=iter(s)
print(isinstance(s_iter,Iterable))
print(isinstance(s_iter,Iterator))

True
False
* * * * * * * * * * * * * * * * * * * * 
True
True


# 生成器
- generator: 一边循环一边计算下一个元素的机制/算法
- 需要满足三个条件：
    - 每次调用都生产出for循环需要的下一个元素或者
    - 如果达到最后一个后，爆出StopIteration异常
    - 可以被next函数调用
- 如何生成一个生成器
    - 直接使用
    - 如果函数中包含yield，则这个函数就叫生成器
    - next调用函数，遇到yield返回

In [11]:
# 直接使用生成器
l=[x*x for x in range(5)]#放在中括号类就是列表生成器
g=(x*x for x in range(5))#放在小括号内就是生成器

print(type(l))
print(type(g))

<class 'list'>
<class 'generator'>


In [13]:
#函数案例
def odd():
    print('step 1')
    print('step 2')
    print('step 3')
    return None

odd()

step 1
step 2
step 3


In [24]:
#生成器的案例
def odd():
    print('step 1')
    yield 1
    print('step 2')
    yield 2
    print('step 3')
    yield 3

    
o =odd()    
one = next(o)
print(one)
two = next(o)
print(two)
three = next(o)
print(three)

step 1
1
step 2
2
step 3
3


In [28]:
#for循环调用生成器
#斐波那契数列案例
def fib(max):
    n,a,b=0,0,1
    while n<max:
        print(b)
        a,b=b,a+b
        n+=1
    return 'Done'
    
fib(7)

1
1
2
3
5
8
13


'Done'

In [33]:
#斐波那契数列案例生成器写法
def fib(max):
    n,a,b=0,0,1
    while n<max:
        yield b
        a,b=b,a+b
        n+=1
    return 'Done'

g=fib(7)
try:
    for i in range(8):
        rst=next(g)
        print(rst)
except StopIteration as e:
    print('结束了')

1
1
2
3
5
8
13
结束了


In [38]:
ge=fib(7)

#最常见的生成器的典型就是range
for i in ge:
    print(i)

1
1
2
3
5
8
13


 # 协程
 - 历史
     - 3.4开始引入,用yield实现
     - 3.5引入协程语法
     - 实现的协程比较好的包有:asyncio,toenado,gevent
- 定义:是为非抢占式多任务产生子程序的计算机程序组件,协程允许不同入口点在不同位置暂停或开始执行程序
- 从技术角度讲,协程就是一个可以暂停的函数,或者直接理解成生成器
- 协程的实现:
    - yield返回
    - send调用
    
- 协程的四个状态:
    - inspect.getgeneratorstate(...)函数确定,该函数会返回下述字符串中的一个:
    - GEN_CREATED：等待开始执行
    - GEN_RUNNING：解释器正在执行
    - GEN_SUSPENED：在yield表达式处暂停
    - GEN_CLOSED：执行结束
    - next预激（prime)
    - 代码案例v2

- 协程终止
    - 协程中未处理的异常会向上冒泡，传给 next 函数或 send 方法的调用方（即触发协程的对象)
    - 终止协程的一种方式：发送某个哨符值，让协程退出。内置的 None 和Ellipsis 等常量经常用作哨符值==
    
- yeild from
    - 调用协程为了得到返回值,协程必须正常终止
    - 生成器正常终止会发出stopIteration异常,异常对象的value属性保存返回值
    - yield from从内部捕获StopIteration异常
    - 案例03
    - 委派生成器
        - 包含yield from表达式的生成器函数
        - 委派生成器在yield from表达式处暂停，调用方可以直接把数据发给子生成器
        - 子生成器在把产出的值发给调用方
        - 自生成器在最后，解释器会抛出StopIteration，并且把返回值附加到异常对象上
        - 案例v04

In [40]:
# 协程代码-1
def simple_coroutine():
    print('->start')
    x=yield
    print('->recived',x)

sc=simple_coroutine()
print(1111)
#可以使用sc.send(None),效果一样
next(sc)#预激


print(2222)
sc.send('zhexiao')

1111
->start
2222
->recived zhexiao


StopIteration: 

In [43]:
# 案例v2,协程的状态
def simple_coroutine(a):
    print('->start')
    b=yield a
    print('->recived',a,b)
    c=yield a+b
    print('->recived',a,b,c)
    
sc=simple_coroutine(5)

aa=next(sc)
print(aa)
bb=sc.send(6)
print(bb)
cc=sc.send(7)
print(cc)

->start
5
->recived 5 6
11
->recived 5 6 7


StopIteration: 

In [46]:
#协程代码案例03
def gen():
    for c in 'AB':
        yield c
        
print(list(gen()))

#类似与通道list连接到'AB'
def gen_new():
    yield from 'AB'

print(list(gen_new()))

['A', 'B']
['A', 'B']


In [6]:
#协程代码案例04, 委派生成器
from collections import namedtuple
'''
解释：
1. 外层 for 循环每次迭代会新建一个 grouper 实例，赋值给 coroutine 变量； grouper 是委派生成器。
2. 调用 next(coroutine)，预激委派生成器 grouper，此时进入 while True 循环，调用子生成器 averager 后，在 yield from 表达式处暂停。
3. 内层 for 循环调用 coroutine.send(value)，直接把值传给子生成器 averager。同时，当前的 grouper 实例（coroutine）在 yield from 表达式处暂停。
4. 内层循环结束后， grouper 实例依旧在 yield from 表达式处暂停，因此， grouper函数定义体中为 results[key] 赋值的语句还没有执行。
5. coroutine.send(None) 终止 averager 子生成器，子生成器抛出 StopIteration 异常并将返回的数据包含在异常对象的value中，yield from 可以直接抓取 StopItration 异常并将异常对象的 value 赋值给 results[key]
'''
ResClass = namedtuple('Res', 'count average')

#子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    
    return ResClass(count, average)


# 委派生成器
def grouper(storages, key):
    while True:
        # 获取averager()返回的值
        storages[key] = yield from averager()
        
# 客户端代码
def client():
    process_data={
        'boys_2': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_1': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }

    storages = {}
    for k, v in process_data.items():

        # 获得协程
        coroutine = grouper(storages, k)

        # 预激协程
        next(coroutine)

        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)


        # 终止协程
        coroutine.send(None)
    print(storages)
    
# run
client()

boys_2 --- [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3]
boys_1 --- [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
{'boys_2': Res(count=9, average=40.422222222222224), 'boys_1': Res(count=9, average=1.3888888888888888)}


In [4]:
import collections
help(collections.namedtuple)

Help on function namedtuple in module collections:

namedtuple(typename, field_names, *, verbose=False, rename=False, module=None)
    Returns a new subclass of tuple with named fields.
    
    >>> Point = namedtuple('Point', ['x', 'y'])
    >>> Point.__doc__                   # docstring for the new class
    'Point(x, y)'
    >>> p = Point(11, y=22)             # instantiate with positional args or keywords
    >>> p[0] + p[1]                     # indexable like a plain tuple
    33
    >>> x, y = p                        # unpack like a regular tuple
    >>> x, y
    (11, 22)
    >>> p.x + p.y                       # fields also accessible by name
    33
    >>> d = p._asdict()                 # convert to a dictionary
    >>> d['x']
    11
    >>> Point(**d)                      # convert from a dictionary
    Point(x=11, y=22)
    >>> p._replace(x=100)               # _replace() is like str.replace() but targets named fields
    Point(x=100, y=22)

